In [167]:
!pip install polars


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [74]:
import polars as pl
pl.Config.set_tbl_rows(100)



polars.config.Config

# Check the data

In [6]:
pl.read_parquet('2.parquet')

timestamp,bid,ask,provider_id
datetime[μs],f64,f64,str
2025-01-01 23:00:00.116069,2620.15,2626.41,"""2"""
2025-01-01 23:00:00.411987,2622.76,2625.98,"""2"""
2025-01-01 23:00:00.432473,2623.21,2625.83,"""2"""
2025-01-01 23:00:00.452675,2623.21,2625.68,"""2"""
2025-01-01 23:00:00.473511,2623.22,2625.64,"""2"""
…,…,…,…
2025-02-03 13:19:59.762306,2810.03,2810.34,"""2"""
2025-02-03 13:19:59.782558,2810.03,2810.29,"""2"""
2025-02-03 13:19:59.802576,2810.02,2810.29,"""2"""


# Data processing

## Union data

In [81]:
df1 = pl.read_parquet("0.parquet")
df2 = pl.read_parquet("1.parquet")
df3 = pl.read_parquet("2.parquet")

merged_df = (
    pl.concat([df1, df2, df3])
    .with_columns([
        ((pl.col("bid") + pl.col("ask")) / 2).alias("mid_price"),
        (pl.col("ask") - pl.col("bid")).alias("spread"),
    ])
)

## Get histogram of spreads

In [53]:
histogram_df = pl.concat(
    [
        (
            df
            .select("spread")
            .to_series()
            .hist(bin_count = 20)
            .with_columns([
                pl.lit(str(provider_id[0])).alias("provider"),
                (pl.col("count") / df.shape[0]).alias("share_of_total_count")
            ])
        )
        for provider_id, df in merged_df.group_by("provider_id", maintain_order=True)
    ]
)

In [59]:
histogram_df.filter(pl.col("share_of_total_count")>0.01)

breakpoint,category,count,provider,share_of_total_count
f64,cat,u32,str,f64
0.25,"""(-0.005, 0.25]""",18964360,"""0""",0.91432
0.5,"""(0.25, 0.5]""",1370310,"""0""",0.066066
6.27,"""(-275.911, 6.27]""",22581824,"""1""",0.999862
1.539,"""(-0.03078, 1.539]""",10585704,"""2""",0.998371


### Analysis
- As per result of this analysis the best source - **provider 0**. The most of the data has spread of -0.005 to 0.5.
- Second place - **provider 2**. Most of the spreads are distributed from -0.03 to 1.539.
- Better to avoid - **provider 1**. Huge spreads, unpredictable behavior.

## Get Data Latency

In [78]:
(
    merged_df
    .sort("timestamp")
    .with_columns([
        pl.col("timestamp").diff().over("provider_id").cast(pl.Duration("ms")).alias("prev_tick_time_diff")
    ])
    .group_by("provider_id")
    .agg([
        pl.col("prev_tick_time_diff").mean().alias("avg_latency"),
        pl.col("prev_tick_time_diff").quantile(0.5).alias("median_latency"),
        pl.col("prev_tick_time_diff").quantile(0.9).alias("p90_latency"),
        pl.col("prev_tick_time_diff").quantile(0.95).alias("p95_latency"),
        pl.col("prev_tick_time_diff").quantile(0.99).alias("p99_latency"),
    ])
)

provider_id,avg_latency,median_latency,p90_latency,p95_latency,p99_latency
str,duration[ms],duration[ms],duration[ms],duration[ms],duration[ms]
"""0""",135ms,9ms,167ms,385ms,1s 388ms
"""2""",265ms,40ms,413ms,777ms,2s 36ms
"""1""",124ms,5ms,156ms,366ms,1s 335ms


### Analysis
- As per result of this analysis the best source - **provider 1**. Average latency - 124ms, median - 5ms. 99 percentile - 1s 335 ms
- Second place - **provider 0**. Almost on par with the **provider 1**, but a bit worse on 11ms in average metric, 4 ms in median metric and 53 ms in P99
- Last and the slowest one - **provider 2**. Almost twice slower in average metric - 265 vs 124 compared with **provider 1**, 4.5x worse than **provider 0** and 8x worse than **provider 1**. P99 is also the worst one.

## Get price volatility

In [85]:
(
    merged_df
    .group_by("provider_id")
    .agg([
        pl.col("mid_price").std().alias("volatility"),
    ])
)

provider_id,volatility
str,f64
"""0""",50.869021
"""1""",51.856656
"""2""",51.343975


### Analysis
- As per result of this analysis the best source - **provider 0**. Average standard deviation - 50.869
- Second place - **provider 2**. 0.4749 points more than **provider 0** 
- Last and the worst one - **provider 1**.

## Get data completeness

In [145]:
expected_rows = pl.DataFrame(pl.datetime_range(
    merged_df["timestamp"].min().replace(microsecond=0),
    merged_df["timestamp"].max().replace(microsecond=0),
    interval="1s",
    eager=True
).alias("timestamps"))


In [164]:
(
    expected_rows
    .join(
        merged_df.select([
            pl.col("timestamp").dt.truncate("1s").alias("timestamp"),
            pl.col("provider_id")
        ]).unique(), left_on = "timestamps", right_on = "timestamp", how = "left"
    )
    .group_by("provider_id")
    .len()
    .with_columns((pl.col("len")/expected_rows.height).alias("share"))
)

provider_id,len,share
str,u32,f64
"""2""",1491239,0.529439
null,1182386,0.419786
"""0""",1462425,0.519209
"""1""",1476359,0.524156


### Analysis
- As per result of this analysis the best source - **provider 2**. Share of existing records - 52.94%
- Second place - **provider 1**. 0.53 percentage points less than **provider 2** 
- Last and the worst one - **provider 0**.
Actually for this check, the difference is not critical, so we shouldn't consider it too much
